# Early Data Exploration
## Joshua Elms
------------------------

This notebook handles some basic parsing and processing for the data.

## Description of Each Cell

1. Necessary imports for the notebook, might require installing these modules if you are attempting to run locally.

2. Handles parsing raw data sources and writing out two CSVs

    * Only needs to be run the first time using this program; after that, you can skip this cell and use the one below

    * Requires you to fill in the paths to each of your input and output sources (relative and absolute paths are both fine)

    * This could take anywhere from 30 seconds to 8-9 minutes, depending on your system

    * The cleaning used here (removing NaNs) is <a href="https://www.analyticsvidhya.com/blog/2020/07/knnimputer-a-robust-way-to-impute-missing-values-using-scikit-learn/#:~:text=The%20idea%20in%20kNN%20methods,neighbors%20found%20in%20the%20dataset."> KNN Imputation </a>. In essence, it finds the closest points to any missing values (calculated by looking at all the values that aren't missing) and fills in the missing based on its nearest neighbor's values.

    * Other methods for this could involve calculating the mean for the entire field and using that, or performing linear regression using the most highly correlated field to the one missing.

3. Using the csv we just wrote, we can load the new dataframe into memory

4. Print a description of summary statistics for each field in the dataframe

5. Generate the correlation matrix for the data - it is exactly the same as the one from your excel file, which is a good confirmation that the data cleaning didn't make the new data unrepresentative of the old

6. This is the start of an example of how I found a list of the most highly correlated variables. We define a function that generates all possible combinations of the input list, and pass into it our list of field names. The time complexity for the next cell is exponential because however many columns we pass to the function here, we will have 2^n combinations to check the correlations of (I ran the complete version of this on BigRed3 to calculate the best ones shown below).

7. Iterate through the list of combinations and perform multilinear regression on each of them, then use the regression results to calculate the correlation between those variables and hailstone size. Every time a new combination is better than the last, it will be printed out for viewing.


## Best Combinations Found on BR3

I excluded a few ranges of variables for the BR3 run; first, columns 1-22 included almost identical versions of 23-33, so I decided not to go for the diminishing returns on accuracy there and just took 1-22 out. Second, of all the columns from 37-54, I only kept 41-50 because the rest were composites that I was attempting to beat. In total, I only used the ranges 21-37 and 41-50, giving 25 variables and 2^25 = 33554432 combinations to check with BR3.

Of these, it returned about 45 new best combinations, and I have included the 3 big jumps in correlation coefficient below.

* Variables SB LI, SB b3km, Shear 0-6 km, and Eff Inflow yield a correlation of 0.2214997567203551
* Variables SB CIN, SB LCL, SB LFC, SB LI, SB hght0c, SB b3km, Shear 0-6 km, Eff Inflow, and SRH 0-3 km yield a correlation of 0.22440901507528213
* Variables SB CIN, SB LCL, SB LFC, SB EL, SB LI, SB hght0c, SB cap, SB b3km, SB brn, Shear 0-1 km, Shear 0-6 km, Eff Inflow, ebwd[0], SRH 0-1 km, SRH 0-3 km, and Eff SRH yield a correlation of 0.22614826662814497

Just from these, we can see it is relatively simple to beat SHIP with only SB Li, SB b3km, Shear 0-6km, and Eff Inflow which have a correlation of ~0.2215 with hailstone size, compared to SHIP's 0.2037.

In [18]:
#1 All modules necessary to run this notebook

import pandas as pd
from itertools import combinations, chain
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from numpy import corrcoef


In [19]:
#2 Parses data sources into usable dataframe, could take a while


raw_data_path = "" # fill in with path to raw data of 53 parameters for each observation
raw_sizes_path = "" # fill in w/ path to sizes of hail
tidy_data_path = "" # fill in path for where you want the tidy (but incomplete, i.e. NaN's present) data
col_names_path = "" # fill w/ path to parameter names file
clean_data_path = "" # fill w/ path to file you want to store clean and complete data in


with open(raw_data_path, "r") as f1:
    with open(raw_sizes_path, "r") as f2:
        with open(tidy_data_path, "w") as f3:
            with open(col_names_path, "r") as f4:
                ### Write Field Names ###
                col_names = [line.strip().rstrip("\n") for line in f4.readlines()]
                col_names.append("Hailstone Size")
                f3.write(",".join(col_names) + "\n")

                ### Read every 53 items, then read size and write all 54 into a output file ###
                pos = 2
                line = f1.readline().strip().rstrip("\n")
                entry_lst = [line]

                while line:                    
                    line = f1.readline().strip().rstrip("\n")
                    entry_lst.append(line)
                    pos += 1

                    if pos == 54:
                        pos = 1
                        entry_lst.append(f2.readline().strip().rstrip("\n"))
                        f3.write(",".join(entry_lst) + "\n")
                        entry_lst = []


# Read tidy csv and clean it up with KNN Imputation, relabel and write out
df = pd.read_csv(tidy_data_path)
df_knn = KNNImputer().fit_transform(df)
df_knn_actual = pd.DataFrame(df_knn)
df_knn_actual.columns = df.columns

df_knn_actual.to_csv(clean_data_path)

FileNotFoundError: [Errno 2] No such file or directory: ''

In [20]:
#3 Load dataframe into memory, display head and tail

path = "/Users/joshuaelms/Desktop/github_repos/CSCI-B365/Meteorology_Modeling_Project/data/pretty_data.csv" # same as clean_data_path, wherever your final csv is

df = pd.read_csv(path, index_col=0)
df.index += 1
df

,ML CAPE,ML CIN,ML LCL,ML LFC,ML EL,ML LI,ML hght0c,ML cap,ML b3km,ML brn,...,lrat,tei,sb_tlcl,ml_tlcl,mu_tlcl,t500,sweat,kinx,crav,Hailstone Size
1,565.886137,-2.456216,591.712340,760.740300,10016.261419,-2.475117,3057.724668,14.031755,164.527269,23.507197,...,6.069679,14.624760,16.369450,14.650496,16.369450,-15.9184,237.167161,27.148344,11784.929088,1.25
2,93.557330,-61.118000,818.659297,1485.730600,4147.988929,1.094013,2878.872717,9.819021,43.581811,2.961764,...,5.573791,11.207180,14.814369,11.829009,14.814369,-15.5371,196.419861,27.202330,1995.924762,1.00
3,416.713894,-0.701233,682.113493,751.489413,7419.731564,-2.174859,3043.083673,14.935360,145.276881,15.405312,...,6.338184,24.258178,15.830423,13.206279,15.830423,-16.9460,195.164206,9.331257,8136.811509,1.00
4,1110.622796,-12.420499,536.926037,989.547800,11364.753475,-4.154931,3532.140768,18.580863,172.484246,22.193236,...,6.614233,19.432405,18.284842,16.262729,17.999692,-15.3177,250.757864,21.326550,31959.336376,1.50
5,1107.162497,-12.514324,536.912773,1008.662600,11386.082876,-4.102513,3583.432806,18.936401,181.416062,23.363115,...,6.578394,21.032620,18.199308,16.380479,18.092846,-15.2050,264.888229,20.604840,32653.287157,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29098,0.000000,0.000000,2343.569759,1107.279086,2343.569759,12.494209,3528.145294,11.552847,0.000000,0.000000,...,7.248768,25.792261,1.717536,-5.193819,7.683926,-15.9886,296.701531,17.164210,0.000000,1.25
29099,0.000000,0.000000,2326.323051,1107.279086,2326.323051,14.986276,3497.386732,11.339111,0.000000,0.000000,...,7.258029,27.763669,0.607784,-7.145827,7.955932,-16.1519,303.557106,19.452570,0.000000,1.00
29100,0.000000,0.000000,2690.384769,2630.432840,2690.384769,14.638317,3482.939445,16.114445,0.000000,0.000000,...,7.280055,25.590055,-0.462974,-9.417577,4.940292,-16.2501,232.337003,11.883790,0.000000,1.00
29101,0.000000,0.000000,2807.261593,2441.488395,2807.261593,16.123160,3451.467575,12.735316,0.000000,0.000000,...,7.248819,25.573522,-0.424774,-11.976938,1.702041,-16.5189,161.945441,6.005967,0.000000,0.88


In [21]:
#4 Prints summary statistics for df, will be way too long to display nicely

df.describe()

,ML CAPE,ML CIN,ML LCL,ML LFC,ML EL,ML LI,ML hght0c,ML cap,ML b3km,ML brn,...,lrat,tei,sb_tlcl,ml_tlcl,mu_tlcl,t500,sweat,kinx,crav,Hailstone Size
count,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,...,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000,29102.000000
mean,1141.143080,-81.896813,1301.060693,2314.606640,9690.948151,-3.514117,4055.743378,16.934669,38.961375,200.843843,...,7.239065,21.247617,14.251640,11.973565,13.677801,-11.858942,303.226112,27.181689,18546.877344,1.233132
std,989.637687,140.670608,552.490866,1128.501556,3429.726589,3.998366,556.103752,4.748191,52.563500,2658.387740,...,0.955675,8.160391,5.331170,5.268721,5.801938,3.316951,122.150754,9.916993,17926.521749,0.521868
min,0.000000,-3509.328383,383.101334,386.799418,396.859210,-15.297868,275.018000,0.000000,0.000000,0.000000,...,2.710508,1.722875,-57.371857,-16.769071,-46.375874,-27.653600,-78.889324,-75.592450,0.000000,0.150000
25%,305.835266,-112.762023,893.117781,1478.504345,8644.974500,-6.035576,3700.433547,14.029655,0.000000,7.776233,...,6.484141,15.242483,11.252429,8.994827,10.684450,-14.112450,214.661880,23.650067,4492.936497,1.000000
50%,958.885288,-40.506631,1191.115297,2261.775381,10711.671530,-3.935588,4109.912039,17.318752,14.817542,30.951001,...,7.206692,20.759785,15.070821,12.945616,14.571887,-11.599050,299.768585,29.655575,13965.183323,1.000000
75%,1748.335956,-4.875115,1618.797002,2970.830400,11972.982532,-1.644113,4467.891729,20.170558,61.211122,83.780950,...,7.974911,26.492111,18.073653,15.811702,17.703021,-9.358648,386.971382,33.623230,27510.411754,1.500000
max,6212.355825,0.000000,4839.833885,11801.322738,15505.246353,28.964380,5578.427290,34.320907,337.432697,220216.101871,...,9.811676,76.567989,26.483992,24.046340,26.483992,-3.176200,706.090889,49.729130,135635.343078,6.000000


In [22]:
#5 Generate the correlation matrix for dataframe, show most strongly correlated variables to Hailstone Size

df_corr = df.corr()

df_corr["Hailstone Size"].abs().sort_values(ascending=False)

Hailstone Size    1.000000
ship              0.203719
crav              0.179000
lrat              0.161617
sweat             0.138882
MU LI             0.137036
SB LI             0.128339
ML LI             0.123399
SB CAPE           0.119698
MU CAPE           0.115961
DCAPE             0.113458
Shear 0-6 km      0.109000
ML CAPE           0.107440
ML EL             0.092197
SB EL             0.090327
stp_fixed         0.087747
MU EL             0.072912
tei               0.072430
SB LCL            0.071378
SB hght0c         0.070903
MU hght0c         0.070903
ML hght0c         0.070903
stp_mixed         0.068026
scp               0.063640
ML LFC            0.056274
ebwd[0]           0.051901
pwat              0.049410
Eff SRH           0.047059
mlmr              0.035647
SB LFC            0.035416
ML LCL            0.032367
ML b3km           0.029507
Shear 0-1 km      0.027687
mu_tlcl           0.027546
MU cap            0.027209
ML CIN            0.026495
MU brn            0.017402
M

In [23]:
#6 Generate the power set (without the empty set) for any given iterator
def power_set(iterable):
    pset = chain.from_iterable(combinations(iterable, r) for r in range(len(iterable)+1))
    return list(list(combo) for combo in pset if len(combo) > 0)



# This section is really just an example; the version I wrote to check all combinations just looks at all 53 columns
# That is a very large set of combinations, so I ran it on BigRed3 for hours to get results
num_cols = 3
field_pset = power_set(df.columns[:num_cols]) # exponential time complexity; 2^num_cols combinations will be generated, so keep it small
field_pset

[['ML CAPE'],
 ['ML CIN'],
 ['ML LCL'],
 ['ML CAPE', 'ML CIN'],
 ['ML CAPE', 'ML LCL'],
 ['ML CIN', 'ML LCL'],
 ['ML CAPE', 'ML CIN', 'ML LCL']]

In [15]:
#7 loop over every combination of powerset, prints the new best combination every time one is discovered
max = 0
for combination in field_pset:
    arr = df[combination].to_numpy()
    target = df["Hailstone Size"].to_numpy()
    obj = LinearRegression().fit(X=arr, y=target)
    coefficients = obj.coef_
    linear_combination = (df[combination]*coefficients).sum(axis=1)
    correlation = corrcoef(linear_combination, target)

    corr = correlation[0][1]

    if corr > max:
        max = corr
        variable_str = f'{", ".join(combination[:-1])}, and {combination[-1]}' if len(combination) > 2 else f'{combination[0]} and {combination[-1]}' if len(combination) == 2 else combination[0]
        print(f"Variable{'s' if len(combination) > 1 else ''} {variable_str} yield{'' if len(combination) > 1 else 's'} a correlation of {corr}")

Variable ML CAPE yields a correlation of 0.10744035377599044
Variable ML LI yields a correlation of 0.12339936047286572
Variables ML CAPE and ML LFC yield a correlation of 0.13397420322247117
Variables ML CAPE and ML b3km yield a correlation of 0.1413263618247371
Variables ML LI and ML b3km yield a correlation of 0.14939146315491544
Variables ML CAPE, ML LI, and ML b3km yield a correlation of 0.1543255941286166
Variables ML CAPE, ML CIN, ML LI, and ML b3km yield a correlation of 0.15664521453989333
Variables ML CAPE, ML LI, ML cap, and ML b3km yield a correlation of 0.1608582833480194
Variables ML CAPE, ML CIN, ML LI, ML cap, and ML b3km yield a correlation of 0.16556220762448348
Variables ML CAPE, ML CIN, ML LCL, ML LI, ML cap, and ML b3km yield a correlation of 0.16707554039400685
Variables ML CAPE, ML CIN, ML LCL, ML LFC, ML LI, ML cap, and ML b3km yield a correlation of 0.16740602037646263
Variables ML CAPE, ML CIN, ML LCL, ML EL, ML LI, ML cap, and ML b3km yield a correlation of 0